In [1]:
# Script to create date for Customer Dimesion Landing

In [2]:
# Import required libraries
import sys
from lib.spark_session import get_spark_session
from lib.utils import date_data, get_string_cols, get_rundate
from lib.job_control import insert_log, get_max_timestamp
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import current_timestamp, lit
from datetime import datetime
from delta import DeltaTable
from lib.aws_s3 import archive_landing_object

In [6]:
# JOB Parameters
rundate = get_rundate()
schema_name = "edw_ld"
table_name = "dim_customer_ld"
table_full_name = f"{schema_name}.{table_name}" 
landing_file_name = f"customer_{rundate}.csv"
landing_file_path = f"s3a://easewithdata-sameer/dw-with-pyspark/landing/customer/{landing_file_name}"
print("SPARK_APP: JOB triggered for rundate - " + rundate)

SPARK_APP: JOB triggered for rundate - 20220101


In [4]:
# Generate Spark Session
spark: SparkSession = get_spark_session(f"Landing load - {table_full_name}")
print("SPARK_APP: Spark UI - " + spark.sparkContext.uiWebUrl)

SPARK_APP: Spark UI - http://358d9e2d6aad:4040


In [5]:
# Spark Configs
spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.set("spark.sql.parquet.mergeSchema", True)

In [7]:
# Create Raw dataframe and get col info
df_raw = spark \
    .read \
    .format("csv") \
    .option("header", True) \
    .load(landing_file_path)

print("SPARK_APP: Printing Raw Schema --")
df_raw.printSchema()

# Get Landing count
print("SPARK_APP: Landing data count - " + str(df_raw.count()))

SPARK_APP: Printing Raw Schema --
root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- email: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- plan_type: string (nullable = true)

SPARK_APP: Landing data count - 18


In [8]:
# Cast all columns to String
df_casted = df_raw.selectExpr(get_string_cols(spark, df_raw))
print("SPARK_APP: Casted all columns to String")

# Add audit columns
df_ld = df_casted.withColumn("insert_dt", current_timestamp()) \
    .withColumn("rundate", lit(rundate))
print("SPARK_APP: Added AUDIT column")

# Get Final Layer count
print("SPARK_APP: Final layer data count - " + str(df_ld.count()))
print("SPARK_APP: Printing Landing Schema --")
df_ld.printSchema()

SPARK_APP: Casted all columns to String
SPARK_APP: Added AUDIT column
SPARK_APP: Final layer data count - 18
SPARK_APP: Printing Landing Schema --
root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- email: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- plan_type: string (nullable = true)
 |-- insert_dt: timestamp (nullable = false)
 |-- rundate: string (nullable = false)



In [9]:
# Write the data to landing layer checking if table exists
if get_max_timestamp(spark, schema_name, table_name) != "1900-01-01 00:00:00.000000":
    df_ld.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(table_full_name)
else:
    df_ld.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable(table_full_name)

print("SPARK_APP: Data written to landing layer")

SPARK_APP: Data written to landing layer


In [10]:
# Archive the landing file
if archive_landing_object(landing_file_name, "customer"):
    print(f"SPARK_APP: Landing file {landing_file_name} archived")
else:
    print(f"SPARK_APP: ERROR - Landing file {landing_file_name} not archived. Please Archive the file manually")

Unable to locate credentials
SPARK_APP: ERROR - Landing file customer_20220101.csv not archived. Please Archive the file manually


In [11]:
# Add job details in JOB CONTROL
insert_log(spark, schema_name, table_name, datetime.now(), rundate)
print("SPARK_APP: Update JOB Control Log")

SPARK_APP: Update JOB Control Log


In [12]:
spark.sql(f"select * from edw.job_control where table_name = '{table_name}' order by insert_dt desc limit 1").show(truncate=False)

+-----------+---------------+--------------------------+--------+--------------------------+
|schema_name|table_name     |max_timestamp             |rundate |insert_dt                 |
+-----------+---------------+--------------------------+--------+--------------------------+
|edw_ld     |dim_customer_ld|2024-04-20 11:52:24.331115|20220101|2024-04-20 11:52:25.879593|
+-----------+---------------+--------------------------+--------+--------------------------+



In [13]:
# Get the logs from delta table version
dt = DeltaTable.forName(spark, table_full_name)
dt.history().limit(1).select("version","operationMetrics.executionTimeMs", 
                                 "operationMetrics.numTargetRowsInserted",
                                "operationMetrics.numTargetRowsUpdated",
                                "operationMetrics.numOutputRows").show(1, False)

+-------+---------------+---------------------+--------------------+-------------+
|version|executionTimeMs|numTargetRowsInserted|numTargetRowsUpdated|numOutputRows|
+-------+---------------+---------------------+--------------------+-------------+
|0      |null           |null                 |null                |18           |
+-------+---------------+---------------------+--------------------+-------------+



In [14]:
# Generate Symlink manifest for Athena Access
dt.generate("symlink_format_manifest")
print("SPARK_APP: Symlink Manifest file generated")

SPARK_APP: Symlink Manifest file generated


In [15]:
spark.stop()